<a href="https://colab.research.google.com/github/Maedeabm/Stroke-Prediction-Odyssey-From-Classic-Classifiers-to-Transformers/blob/main/Fraud_Detection_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fraud detection model using a standard neural network approach

Step 1: Data Preprocessing

In [34]:
# Required Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn

# Load data
data = pd.read_csv('creditcard.csv')  # Assumes you've loaded the dataset

# Split the data
X = data.drop(columns=['Class'])
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Balance classes using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Convert to PyTorch tensors
X_train_resampled = torch.tensor(X_train_resampled, dtype=torch.float32)
y_train_resampled = torch.tensor(y_train_resampled.values, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)


2. Define the Neural Network:

In [35]:
class FraudDetectionModel(nn.Module):
    def __init__(self, input_dim):
        super(FraudDetectionModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        return torch.sigmoid(self.layer4(x))

model = FraudDetectionModel(input_dim=X_train_resampled.shape[1])


3. Training the Model:

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 15

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_resampled.clone().detach().to(device))
    loss = criterion(outputs, y_train_resampled.clone().detach().to(device))

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/15, Loss: 0.6979
Epoch 2/15, Loss: 0.6603
Epoch 3/15, Loss: 0.6350
Epoch 4/15, Loss: 0.6173
Epoch 5/15, Loss: 0.6005
Epoch 6/15, Loss: 0.5837
Epoch 7/15, Loss: 0.5666
Epoch 8/15, Loss: 0.5491
Epoch 9/15, Loss: 0.5313
Epoch 10/15, Loss: 0.5132
Epoch 11/15, Loss: 0.4949
Epoch 12/15, Loss: 0.4766
Epoch 13/15, Loss: 0.4584
Epoch 14/15, Loss: 0.4404
Epoch 15/15, Loss: 0.4227


4. Evaluating the Model:

In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

model.eval()
with torch.no_grad():
    y_pred = model(torch.tensor(X_test, dtype=torch.float32).to(device))
    y_pred_class = (y_pred > 0.5).float().cpu()

    accuracy = accuracy_score(y_test, y_pred_class)
    precision = precision_score(y_test, y_pred_class)
    recall = recall_score(y_test, y_pred_class)
    f1 = f1_score(y_test, y_pred_class)
    roc_auc = roc_auc_score(y_test, y_pred.cpu())

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")


<ipython-input-40-6c5bdad8e568>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = model(torch.tensor(X_test, dtype=torch.float32).to(device))


Accuracy: 0.9816
Precision: 0.0775
Recall: 0.8878
F1 Score: 0.1426
ROC AUC Score: 0.9338
